# 向量数据库介绍和使用

## 1. 向量数据库简介

In [1]:
import sys
sys.path.append("../")

In [2]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.embeddings import QianfanEmbeddingsEndpoint
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from QA_Project.project.embedding.zhipuai_embedding import ZhipuAIEmbeddings
from langchain_community.llms import QianfanLLMEndpoint
from langchain.llms import HuggingFacePipeline
from QA_Project.project.llm.spark_llm import Spark_LLM

In [3]:
import os
os.environ['HTTPS_PROXY'] = 'http://192.168.8.94:10809'
os.environ["HTTP_PROXY"] = 'http://192.168.8.94:10809'

In [4]:
# 使用前配置自己的 api 到环境变量中如
import sys
from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())
_ = load_dotenv('../QA_Project/.env')

# 获取环境变量 
wenxin_api_key = os.environ["wenxin_api_key"]
wenxin_secret_key = os.environ["wenxin_secret_key"]
spark_app_id = os.environ["spark_app_id"]
spark_api_key = os.environ["spark_api_key"]
spark_secret_key = os.environ["spark_secret_key"]
zhipuai_api_key = os.environ["ZHIPUAI_API_KEY"]

In [5]:
# 加载 PDF
loaders_chinese = [
    PyMuPDFLoader("../QA_Project/data_base/knowledge_db/pumpkin_book/pumpkin_book.pdf") # 南瓜书
    # 大家可以自行加入其他文件
]
docs = []
for loader in loaders_chinese:
    docs.extend(loader.load())
# 切分文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
split_docs = text_splitter.split_documents(docs)


# 定义 Embeddings
qianfan_embedding = QianfanEmbeddingsEndpoint(qianfan_ak=wenxin_api_key,
                                  qianfan_sk=wenxin_secret_key) 
# embedding = HuggingFaceEmbeddings(model_name="moka-ai/m3e-base", model_kwargs=model_kwargs)
# spark_embedding = SparkLLMTextEmbeddings(spark_app_id=spark_app_id, spark_api_key=spark_api_key, spark_api_secret=spark_secret_key)

In [6]:
persist_directory = '../QA_Project/data_base/vector_db/test_chroma'

In [ ]:
# !rm -rf '../QA_Project/data_base/vector_db/test_chroma'  # 删除旧的数据库文件（如果文件夹中有文件的话），window电脑请手动删除

## 2. 构建chroma向量库

In [7]:
vectordb = Chroma.from_documents(
    documents=split_docs[:100], # 为了速度，只选择了前 100 个切分的 doc 进行生成。
    embedding=qianfan_embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

[INFO] [03-04 11:49:33] openapi_requestor.py:316 [t:140278941804352]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-04 11:49:33] oauth.py:207 [t:140278941804352]: trying to refresh access_token for ak `NSwW3Z***`
[INFO] [03-04 11:49:34] oauth.py:220 [t:140278941804352]: sucessfully refresh access_token
[INFO] [03-04 11:49:34] openapi_requestor.py:316 [t:140278941804352]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-04 11:49:35] openapi_requestor.py:316 [t:140278941804352]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-04 11:49:35] openapi_requestor.py:316 [t:140278941804352]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-04 11:49:36] openapi_requestor.py:316 [t:140278941804352]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-04 11:49:37] openapi_requestor.py:316 [t:140278941804352]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-04 11:49:37] openapi_requestor.py:316 [t:14027

In [8]:
vectordb.persist()

In [9]:
# 加载已经构建好的向量数据库
# vectordb = Chroma(
#     persist_directory=persist_directory,
#     embedding_function=embedding
# )

In [10]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：100


## 3. 通过向量数据库检索

### 3.1 相似度检索

In [11]:
question="什么是机器学习"

In [12]:
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

[INFO] [03-05 09:55:01] openapi_requestor.py:316 [t:140511338940224]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-05 09:55:01] oauth.py:207 [t:140511338940224]: trying to refresh access_token for ak `NSwW3Z***`
[INFO] [03-05 09:55:01] oauth.py:220 [t:140511338940224]: sucessfully refresh access_token


检索到的内容数：3


In [13]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

检索到的第0个内容: 
前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我
--------------
检索到的第1个内容: 
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》
←_←
第 12 章 计算学习理论
136
12.1 基础知识
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
136
12.1.1 式 (12.1) 的解释
. . . . . . . . . . . 
--------------
检索到的第2个内容: 
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》
←_←
目录
第 1 章 绪论
1
1.1
引言 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
1
1.2
基本术语
. . . . . . . . . . . . . . . . . . 
--------------


### 3.2 MMR检索

In [14]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

[INFO] [03-05 09:55:16] openapi_requestor.py:316 [t:140511338940224]: requesting llm api endpoint: /embeddings/embedding-v1


In [15]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

MMR 检索到的第0个内容: 
前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我
--------------
MMR 检索到的第1个内容: 
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》
←_←
8.2.15 式 (8.19) 的推导
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
82
8.2.16 AdaBoost 的个人推导 . . . . . . . . . . . . . . . . . . . . . . .
--------------
MMR 检索到的第2个内容: 
45
5.5.3
式 (5.22) 的解释
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
45
5.5.4
式 (5.23) 的解释
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
45
5.6
深
--------------


## 4.构造检索式问答链

### 4.1 直接询问LLM

In [16]:
# 导入检索式问答链
from langchain.chains import RetrievalQA

In [17]:
llm = QianfanLLMEndpoint(qianfan_ak=wenxin_api_key,
                                  qianfan_sk=wenxin_secret_key,temperature=0.1)

In [18]:
# # 可以使用 HuggingFacePipeline 本地搭建大语言模型
# model_id = 'THUDM/chatglm2-6b-int4' # 采用 int 量化后的模型可以节省硬盘占用以及实时量化所需的运算资源
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModel.from_pretrained(model_id, trust_remote_code=True).half().quantize(4).cuda()
# model = model.eval()
# pipe = pipeline(
#     "text2text-generation",
#     model=model, 
#     tokenizer=tokenizer, 
#     max_length=100
# )

# llm = HuggingFacePipeline(pipeline=pipe)
# # 

In [18]:
# 声明一个检索式问答链
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [20]:
# 可以以该方式进行检索问答
question = "本知识库主要包含什么内容"
result = qa_chain({"query": question})
print(f"大语言模型的回答为：{result['result']}")


[INFO] [03-05 09:55:41] openapi_requestor.py:316 [t:140511338940224]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-05 09:55:42] openapi_requestor.py:316 [t:140511338940224]: requesting llm api endpoint: /chat/eb-instant


大语言模型的回答为：本知识库主要包含机器学习的基础知识，包括深度学习、层次聚类、降维与度量学习、特征选择与稀疏学习等内容。此外，还包含一些预备知识和符号约定。


### 4.2 结合prompt的提问

In [21]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """使用以下上下文片段来回答最后的问题。如果你不知道答案，只需说不知道，不要试图编造答案。答案最多使用三个句子。尽量简明扼要地回答。在回答的最后一定要说"感谢您的提问！"
{context}
问题：{question}
有用的回答："""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [22]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


In [23]:
question = " 2025 年大语言模型效果最好的是哪个模型"

In [24]:
result = qa_chain({"query": question})
print(f"LLM 对问题的回答：{result['result']}")

[INFO] [03-05 09:55:59] openapi_requestor.py:316 [t:140511338940224]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-05 09:56:00] openapi_requestor.py:316 [t:140511338940224]: requesting llm api endpoint: /chat/eb-instant


LLM 对问题的回答：根据给出的上下文片段，无法确定2025年大语言模型效果最好的是哪个模型。需要更多的信息才能回答这个问题。


In [25]:
print(f"向量数据库检索到的最相关的文档：{result['source_documents'][0]}")

向量数据库检索到的最相关的文档：page_content='15\n2.5.1\n式 (2.37) 到式 (2.42) 的推导 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n15\n第 3 章 线性模型\n18\n3.1\n基本形式\n. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n18\n3.2\n线性回归\n. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n18\n3.2.1\n属性数值化 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n18\n3.2.2' metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': '../QA_Project/data_base/knowledge_db/pumpkin_book/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 2, 'producer': 'xdvipdfmx (20200315)', 'source': '../QA_Project/data_base/knowledge_db/pumpkin_book/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}


In [30]:
qa_chain2 = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type="map_reduce"
)

In [31]:
result = qa_chain2({"query": question})
print(f"LLM 对问题的回答：{result['result']}")

[INFO] [03-05 09:59:49] openapi_requestor.py:316 [t:140511338940224]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-05 09:59:50] openapi_requestor.py:316 [t:140511338940224]: requesting llm api endpoint: /chat/eb-instant
[INFO] [03-05 09:59:52] openapi_requestor.py:316 [t:140511338940224]: requesting llm api endpoint: /chat/eb-instant
[INFO] [03-05 09:59:54] openapi_requestor.py:316 [t:140511338940224]: requesting llm api endpoint: /chat/eb-instant
[INFO] [03-05 09:59:56] openapi_requestor.py:316 [t:140511338940224]: requesting llm api endpoint: /chat/eb-instant
[INFO] [03-05 09:59:57] openapi_requestor.py:316 [t:140511338940224]: requesting llm api endpoint: /chat/eb-instant


LLM 对问题的回答：无法回答。这段文字并未提及关于2025年大语言模型效果最好的是哪个模型的具体信息。


In [32]:
print(f"向量数据库检索到的最相关的文档：{result['source_documents'][0]}")

向量数据库检索到的最相关的文档：page_content='15\n2.5.1\n式 (2.37) 到式 (2.42) 的推导 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n15\n第 3 章 线性模型\n18\n3.1\n基本形式\n. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n18\n3.2\n线性回归\n. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n18\n3.2.1\n属性数值化 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n18\n3.2.2' metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': '../QA_Project/data_base/knowledge_db/pumpkin_book/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 2, 'producer': 'xdvipdfmx (20200315)', 'source': '../QA_Project/data_base/knowledge_db/pumpkin_book/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}
